In [4]:
# Installing necessary libraries and dependencies

from tracking.sort import *
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from utils import get_car,read_license_plate,write_csv
from PIL import Image
%matplotlib inline


AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [4]:
pip install -r requirements.txt

  Using cached ultralytics-8.0.200-py3-none-any.whl.metadata (31 kB)
  Using cached pandas-2.0.2-cp310-cp310-win_amd64.whl.metadata (18 kB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached numpy-1.24.3-cp310-cp310-win_amd64.whl (14.8 MB)
  Using cached scipy-1.10.1-cp310-cp310-win_amd64.whl (42.5 MB)
  Using cached easyocr-1.7.0-py3-none-any.whl.metadata (14 kB)
  Using cached filterpy-1.4.5-py3-none-any.whl
  Using cached lap-0.4.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Pillow-9.5.0-cp310-cp310-win_amd64.whl (2.5 MB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached seaborn-0.13.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scikit_image-0.22.0-cp310-cp310-win_amd64.whl.met

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [52 lines of output]
      C:\Users\PyAlgo\AppData\Local\Temp\pip-install-urrki_ah\lap_fba67a61f61848f690214756cb4fcee0\setup.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import parse_version
      Partial import of lap during the build process.
      C:\Users\PyAlgo\AppData\Local\Temp\pip-install-urrki_ah\lap_fba67a61f61848f690214756cb4fcee0\setup.py:223: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils

In [ ]:

#loading Trained Model for vehicle detection/plate detection/vehicle tracking

vehicle_detect_model=YOLO('./vehicle_detection/models/yolov8n.pt')

number_plate_detect_model=YOLO('./number_plate_detection/models/best.pt')

vehicle_tracking_model=Sort()

#loading video file (mention path) / real time mention (0 in cv2.VideoCapture)

video=cv2.VideoCapture('./videos/sample3.mp4')


results = {}
ret=True
frame_no=-1
vehicle_classid=[0,1,2,3,4]

while ret:
    ret,frame=video.read()
    if ret:
        results[frame_no] = {}
        # first detecting vehicles and getting bounding boxes
        vehicle_detection=vehicle_detect_model(frame)[0]
        detection_=[]
        for detection in vehicle_detection.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection

            if class_id in vehicle_classid:
                detection_.append([x1,y1,x2,y2,score])
        

        ###### To visualise the detection of vehicles from various frame ######

            # if score>0.3:
            #     detection_.append(((int(x1),int(y1)),(int(x2),int(y2))))

        # for i in detection_:
        #     cv2.rectangle(frame,i[0],i[1],(0,255,0))
        # cv2.imshow('test',frame)


        #######################################################################

        ###  tracking vehicles ####
        
        vehicle_tracking=vehicle_tracking_model.update(np.asarray(detection_))

        ## second detecting number plates and getting bounding boxes


        number_plate_detection=number_plate_detect_model(frame)[0]
        for license_plate in number_plate_detection.boxes.data.tolist():    
            x1, y1, x2, y2, score, class_id = license_plate
        

        #### To visualise the detection of number plates from various frame ######
        #     cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0))
        # cv2.imshow('test',frame)
        # cv2.waitKey(1)
        # cv2.destroyAllWindows()
        # break

        ######################################################################
       


            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, vehicle_tracking)
            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 120, 255, cv2.THRESH_BINARY_INV)
                
                

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)


                ####### To visualise the detection of number plates and vehicles from various frame ######

                cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),thickness=5)
                cv2.rectangle(frame,(int(xcar1),int(ycar1)),(int(xcar2),int(ycar2)),(0,255,0),thickness=5)
                cv2.imshow('output',frame)
            



                # # if license_plate_text_score !=[]:
                # results[frame_no][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                #                                 'license_plate': {'bbox': [x1, y1, x2, y2],
                #                                                 'text': license_plate_text,
                #                                                 'bbox_score': score,
                #                                                 'text_score': license_plate_text_score}}
                
    
        cv2.waitKey(1)
        

        frame_no+=1

#write_csv(results, './test.csv')

cv2.destroyAllWindows()
video.release()



        

    

        



 


0: 384x640 1 person, 1 car, 1 bus, 3 trucks, 62.0ms
Speed: 2.3ms preprocess, 62.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license, 59.0ms
Speed: 2.0ms preprocess, 59.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 bus, 2 trucks, 65.9ms
Speed: 1.8ms preprocess, 65.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license, 197.7ms
Speed: 5.3ms preprocess, 197.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 59.9ms
Speed: 1.5ms preprocess, 59.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 75.6ms
Speed: 2.7ms preprocess, 75.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 81.2ms
Speed: 1.9ms preprocess, 81.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.

In [8]:
import torch
torch.cuda.current_device()

0